In [1]:
import networkx as nx
import random
import numpy as np
import itertools
from time import perf_counter

In [2]:
n = 300

In [3]:
G = nx.complete_graph(n, nx.DiGraph())
for (u,v) in G.edges:
    G.edges[u,v]['weight'] = random.random()

### Approximating PathMaxATSP via matching

In [4]:
start_perf = perf_counter()

In [5]:
Gu = nx.complete_graph(n)
for (u,v) in Gu.edges:
    w1 = G.edges[u,v]['weight']
    w2 = G.edges[v,u]['weight']
    Gu.edges[u,v]['weight'] = max(w1, w2)

In [6]:
matching = nx.max_weight_matching(Gu)

In [7]:
dir_matching = []
for (u,v) in matching:
    if G.edges[u,v]['weight'] > G.edges[v,u]['weight']:
        dir_matching.append((u,v))
    else:
        dir_matching.append((v,u))

In [8]:
def draw_edge_set(Gu, edges):
    colors = []
    for (u,v) in Gu.edges:
        if (u,v) in edges or (v,u) in edges:
            colors.append("red")
        else:
            colors.append("black")

    nx.draw_networkx(Gu, edge_color=colors)

In [9]:
# draw_edge_set(Gu, dir_matching)

In [10]:
def edge_set_to_path(edges):
    path = []
    for (u,v) in dir_matching:
        path.append(u)
        path.append(v)
    if n % 2 == 1:
        vertex_sum = n * (n-1) / 2
        final_v = int(vertex_sum - sum(path))
        path.append(final_v)
    return path

In [11]:
path1 = edge_set_to_path(dir_matching)

In [12]:
end_perf = perf_counter()
path1_time = end_perf - start_perf

In [13]:
def get_path_weight(path):
    total_weight = 0
    for u,v in zip(path, path[1:]):
        total_weight += G.edges[u,v]['weight']
    return total_weight

In [14]:
# print(get_path_weight(path1))
# print(path1)

In [15]:
# draw_edge_set(Gu, list(zip(path1, path1[1:])))

### Approximating PathMaxATSP via cycle cover

WARNING: NetworkX uses an assignment problem solver other than the Hungarian method

In [16]:
from networkx.algorithms import bipartite

In [17]:
start_perf = perf_counter()

In [18]:
Gb = nx.Graph()
for i in range(2*n):
    Gb.add_node(i)
for (u,v) in G.edges:
    Gb.add_edge(u, v+n, weight=1-G.edges[u,v]['weight'])  # account for min weight NetworkX method

In [19]:
mb_dict = bipartite.minimum_weight_full_matching(Gb, top_nodes=list(Gb.nodes)[:n])

In [20]:
# draw_edge_set(Gb, list(mb_dict.items()))

In [21]:
m_dict = {}
for u,v in mb_dict.items():
    if u < n:
        x,y = u,v
    else:
        x,y = v,u
    m_dict[x] = y-n

In [22]:
# draw_edge_set(Gu, list(m_dict.items()))

In [23]:
cycle_cover = []

done = set()

cycle = []
start = curr = 0

while True:
    cycle.append(curr)
    done.add(curr)
    curr = m_dict[curr]
    if curr == start:
        cycle_cover.append(cycle)
        cycle = []
        flag = False
        for i in range(n):
            if i not in done:
                start = curr = i
                flag = True
                break
        if not flag:
            break

In [24]:
path_cover = []
for cycle in cycle_cover:
    best_start = cycle[0]
    best_start_idx = 0
    min_weight = G.edges[cycle[-1],cycle[0]]['weight']  # all weights are in [0,1]
    for i, (u,v) in enumerate(zip(cycle, cycle[1:])):
        if G.edges[u,v]['weight'] < min_weight:
            best_start = v
            best_start_idx = i+1
            min_weight = G.edges[u,v]['weight']
    path = cycle[best_start_idx:] + cycle[:best_start_idx]
    path_cover.append(path)

In [25]:
path3 = list(itertools.chain(*path_cover))

In [26]:
end_perf = perf_counter()
path3_time = end_perf - start_perf

In [27]:
# print(get_path_weight(path3))
# print(path3)

In [28]:
# draw_edge_set(Gu, list(zip(path3, path3[1:])))

## Results

#### expected random path weight

In [29]:
(n-1)*0.5

149.5

#### theoretical upper bound on path weight

In [30]:
(n-1)*1

299

#### matching approx

In [31]:
print(f"weight {get_path_weight(path1)} in {path1_time} seconds")

weight 228.44499891030858 in 35.998076899995795 seconds


#### cycle cover approx

In [32]:
print(f"weight {get_path_weight(path3)} in {path3_time} seconds")

weight 295.7804644362829 in 1.2505307999963406 seconds
